In [500]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import regex

In [501]:
URL =  "https://www.psychologytoday.com/us/therapists/or/portland/714186?sid=6109d19cf1992&ref=2"

In [502]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}


In [503]:
page = requests.get(URL, headers=headers)

In [504]:
soup = BeautifulSoup(page.content, "html.parser")

In [505]:
zip_code = soup.find(itemprop="postalcode").get_text()

In [506]:
name = soup.find(itemprop="name").get_text()

In [507]:
title = soup.find("h2").get_text()

In [508]:
specialties = soup.find("ul", class_="attribute-list specialties-list").get_text()

In [509]:
insurance = soup.find("ul", class_= "attribute-list copy-small").get_text()

In [510]:
price = soup.find("div", class_="finances-office").get_text()#.contents

In [511]:
price = price.split('\n')

In [512]:
while '' in price:
    price.remove('')

In [513]:
records = []
next_url = "https://www.psychologytoday.com/us/therapists/or/portland/708685?sid=6109d19cf1992&ref=1"
for person in range(10):

    next_page = requests.get(next_url, headers=headers)
    next_soup = BeautifulSoup(next_page.content, "html.parser")
    next_zip_code = next_soup.find(itemprop="postalcode").get_text()
    next_name = next_soup.find(itemprop="name").get_text()
    next_title = next_soup.find("h2").get_text()
    next_specialties = next_soup.find("ul", class_="attribute-list specialties-list").get_text()
    # next_insurance = next_soup.find("ul", class_= "attribute-list copy-small").get_text()
    next_price = next_soup.find("div", class_="finances-office").get_text().split('\n')
    while '' in next_price:
        next_price.remove('')
    while '            ' in next_price:
        next_price.remove('            ')
    while '                                                    ' in next_price:
        next_price.remove('                                                    ')
    price = next_price[0]
    records.append((next_name.strip(), next_zip_code, next_title, next_specialties, price))

    # print(next_name.strip())
    # print(next_zip_code)
    # print(next_specialties)
    # print(next_title.strip())
    # #print(insurance)
    # print(next_price)
    #time.sleep(1)
    next_url = next_soup.find("a", class_="profile-next")["href"]
#print(records)

In [514]:
next_price

['Cost per Session: $90 - $120',
 'Sliding Scale: Yes',
 '                            I cannot accept insurance; however, I am happy to provide documentation so you may receive reimbursement, if you insurance allows.',
 '                ',
 'Email me or call (971) 717-3995 with questions.',
 '                                    ']

In [515]:
df = pd.DataFrame(records, columns=["name","zip","title","specialties","price"])


In [516]:
df["specialties"] = df["specialties"].str.replace(' ','')

In [517]:
df[["title","specialties","price"]] = df[["title","specialties","price"]].replace('\n',' ', regex=True)

In [518]:
df["price"] = df["price"].str.replace('Cost per Session: ','')

In [519]:
df["title"] = df["title"].str.replace(' ','')

In [520]:
df

,name,zip,title,specialties,price
0,Kierstin Leigh Oliver,97220,"LicensedProfessionalCounselor,MS,LCPC,LPC",Anxiety Women'sIssues RelationshipIssues,$50 - $125
1,Rachel Archer,97213,"Psychologist,PsyD",Anxiety Depression,$205 - $215
2,James Gibb Counseling,97229,"LicensedProfessionalCounselor,MA,LPC,NCC,BC-TM...",AngerManagement Anxiety Depression,$60 - $150
3,Robert Markell,97213,"LicensedProfessionalCounselor,LMHC,LPC",,$120 - $160
4,LeAnna Mill,97216,"LPCIntern,MALPC",DomesticAbuse Anxiety Depression,$40 - $100
5,Cherrie Padua,97217,"LicensedProfessionalCounselor,MA,LPC",Anxiety Depression Stress,$100 - $150
6,John Lenkin,97209,"LPCIntern,SEP",,$25 - $70
7,Jennifer Ripley-Wood,97202,"LicensedProfessionalCounselor,LPC",Anxiety Depression TraumaandPTSD,$140 - $160
8,Ann Marie Miner,97214,"Psychologist,PsyD","Pregnancy,Prenatal,Postpartum Relationship...",$160 - $180
9,Sylas L Veit,97214,LPCIntern,Transgender RelationshipIssues Anxiety,$90 - $120
